In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd

file_path = '/content/drive/My Drive/Diploma/Jan 2025 Term/BDM Project/Data/Income from Washing Ironing Oct-Dec 24.xlsx'
df = pd.read_excel(file_path)

df.columns = [col.strip() for col in df.columns]

# Extract prefix from Room No and normalize for faculty block types
df['Prefix'] = df['Room No'].astype(str).str.split(' ').str[0]
faculty_blocks_base = ['C', 'D', 'P', 'E', 'F', 'H', 'P.H.D', 'OUTSIDE']
df['Normalized Prefix'] = df['Prefix'].apply(
    lambda x: ''.join([char for char in x if not char.isdigit()]) if any(x.startswith(fac) for fac in faculty_blocks_base) else x)

# Classify as Student or Faculty
student_prefixes = ['VM', 'VK', 'MM', 'G', 'GT', 'V', 'S', 'B', 'K', 'GG', 'M', 'R']
df['Type'] = df['Normalized Prefix'].apply(lambda x: 'Student' if x in student_prefixes else 'Faculty')

student_df = df[df['Type'] == 'Student']
faculty_df = df[df['Type'] == 'Faculty']

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
print(student_df)

           Date Room No  Amount Prefix Normalized Prefix     Type
0    2024-10-01   GG 56    80.0     GG                GG  Student
1    2024-10-01  VM 388    68.0     VM                VM  Student
2    2024-10-01  VM 491    75.0     VM                VM  Student
3    2024-10-01   B 310    82.0      B                 B  Student
4    2024-10-01  VM 456   110.0     VM                VM  Student
...         ...     ...     ...    ...               ...      ...
5080 2024-12-31   B 140   147.0      B                 B  Student
5081 2024-12-31   K 342    80.0      K                 K  Student
5082 2024-12-31  VM 416   150.0     VM                VM  Student
5083 2024-12-31  VM 368    90.0     VM                VM  Student
5084 2024-12-31   V 375   150.0      V                 V  Student

[4904 rows x 6 columns]


In [ ]:
print(faculty_df)

           Date  Room No  Amount   Prefix Normalized Prefix     Type
87   2024-10-04    P.H.D   135.0    P.H.D             P.H.D  Faculty
95   2024-10-05    P 136    46.0        P                 P  Faculty
104  2024-10-05  OUTSIDE    90.0  OUTSIDE           OUTSIDE  Faculty
198  2024-10-06     D3 3   360.0       D3                 D  Faculty
203  2024-10-06    P.H.D   169.0    P.H.D             P.H.D  Faculty
...         ...      ...     ...      ...               ...      ...
5039 2024-12-27     E5 2  1325.0       E5                 E  Faculty
5043 2024-12-27   P3 803   192.0       P3                 P  Faculty
5044 2024-12-27   P3 804   133.0       P3                 P  Faculty
5059 2024-12-29   H8 101   256.0       H8                 H  Faculty
5064 2024-12-29     E9 6   210.0       E9                 E  Faculty

[181 rows x 6 columns]


In [13]:
# Student block summary
student_summary = student_df.groupby('Normalized Prefix').agg(
    Total_Amount=('Amount', 'sum'),
    Total_Transactions=('Room No', 'count')
).reset_index()

print("Student Block Summary:")
print(student_summary)

Student Block Summary:
   Normalized Prefix  Total_Amount  Total_Transactions
0                  B       34426.0                 326
1                  G       44295.0                 403
2                 GG       20731.0                 188
3                 GT       43065.0                 383
4                  K       32387.0                 290
5                  M       21284.0                 159
6                 MM       48179.0                 404
7                  R       17226.0                 151
8                  S       39418.0                 337
9                  V       38413.0                 374
10                VK       92974.0                 823
11                VM      116109.0                1066


In [14]:
# Faculty block summary
faculty_summary = faculty_df.groupby('Normalized Prefix').agg(
    Total_Amount=('Amount', 'sum'),
    Total_Transactions=('Room No', 'count')
).reset_index()

print("\nFaculty Block Summary:")
print(faculty_summary)


Faculty Block Summary:
   Normalized Prefix  Total_Amount  Total_Transactions
0                  C         872.0                   7
1                 CP         344.0                   2
2                  D        6513.0                  21
3                  E        2664.0                   7
4                  F         534.0                   4
5                  H       14290.0                  81
6                  J         112.0                   1
7            OUTSIDE         748.0                   5
8                  P        1341.0                  11
9              P.H.D        8520.0                  40
10               nan           0.0                   0


In [ ]:
from sklearn.cluster import KMeans

# function for clustering input
def prepare_cluster_input(data):
    return data.groupby('Normalized Prefix').agg({
        'Amount': 'sum',
        'Room No': 'count'
    }).rename(columns={'Room No': 'Frequency'}).reset_index()

# Prepare and cluster student data
student_cluster_input = prepare_cluster_input(student_df)
kmeans_student = KMeans(n_clusters=3, random_state=42)
student_cluster_input['Cluster'] = kmeans_student.fit_predict(student_cluster_input[['Amount', 'Frequency']])

# Prepare and cluster faculty data
faculty_cluster_input = prepare_cluster_input(faculty_df)
kmeans_faculty = KMeans(n_clusters=3, random_state=42)
faculty_cluster_input['Cluster'] = kmeans_faculty.fit_predict(faculty_cluster_input[['Amount', 'Frequency']])

In [ ]:
print("Student Clusters:")
print(student_cluster_input)

Student Clusters:
   Normalized Prefix    Amount  Frequency  Cluster
0                  B   34426.0        326        2
1                  G   44295.0        403        2
2                 GG   20731.0        188        0
3                 GT   43065.0        383        2
4                  K   32387.0        290        2
5                  M   21284.0        159        0
6                 MM   48179.0        404        2
7                  R   17226.0        151        0
8                  S   39418.0        337        2
9                  V   38413.0        374        2
10                VK   92974.0        823        1
11                VM  116109.0       1066        1


In [ ]:
print("\nFaculty Clusters:")
print(faculty_cluster_input)


Faculty Clusters:
   Normalized Prefix   Amount  Frequency  Cluster
0                  C    872.0          7        0
1                 CP    344.0          2        0
2                  D   6513.0         21        1
3                  E   2664.0          7        2
4                  F    534.0          4        0
5                  H  14290.0         81        1
6                  J    112.0          1        0
7            OUTSIDE    748.0          5        0
8                  P   1341.0         11        0
9              P.H.D   8520.0         40        1
10               nan      0.0          0        0
